In [83]:
import pandas as pd
import numpy as np
import os

import matplotlib.pyplot as plt
%matplotlib inline

CWD = "/".join(os.getcwd().split("/")[:-2])

In [84]:
df = pd.read_csv(f"{CWD}/analysis/results/mcmc/mcmcoutput.csv")
df = df.loc[[0,2,4]]
df.theta_a_p50,df.theta_a_p16,

(0     73.121624237605
 2    31.7545466783349
 4    46.8294956847332
 Name: theta_a_p50, dtype: object, 0    71.6583396311642
 2    25.3737134205418
 4     43.693619297167
 Name: theta_a_p16, dtype: object)

In [85]:
def add_val_with_percentiles(df, val, out):

    suff = ["_p16","_p50","_p84"]
    for s in suff:
        df[val+s] = df[val+s].apply(lambda x: float(x))
    
    df["vplus"] = df.apply(lambda x: x[val+suff[2]]-x[val+suff[1]], axis=1)
    df["vminus"] = df.apply(lambda x: x[val+suff[1]]-x[val+suff[0]], axis=1)
    power = int(np.round(np.log10(np.abs(float(np.min(df.vplus))))))
    print(power, "POWER")
    powerabs = int(np.round(np.log10(np.abs(float(np.min( df[val+suff[1]]))))))
    print(powerabs, "POWERABS")
    if abs(powerabs) > 5:
        out = out + f"$\cdot 10^{powerabs}"
        out = out.replace("^","^{")
        out = out + "}$"
        df.vminus = df.vminus/(10**(power))
        df.vplus = df.vplus/(10**(power))
        df[val+suff[1]] = df[val+suff[1]]/(10**powerabs)
        
        df[out] = df.apply(lambda x: f"${x[val+suff[1]]:.{powerabs-power}f}\left(^{x.vplus:.0f}_{x.vminus:.0f}\right)$", axis=1)
        df[out] = df[out].apply(lambda x: x.replace("^","^{").replace("_","}_{").replace("\right)","}\right)"))
    else:
        print(df.vplus/(10**power))
        if (df.vplus/(10**power)<1).any():
            power -= 1
            print("TRUE")
        if powerabs - power < 0:
            raise ValueError("uncertainty larger than 50th percentile")
        if power < 0: r = abs(power)
        elif power >= 0: r = 0
        df[out] = df.apply(lambda x: f"${x[val+suff[1]]:.{r}f}\left(^{x.vplus/(10**power):.0f}_{x.vminus/(10**power):.0f}\right)$", axis=1)
        df[out] = df[out].apply(lambda x: x.replace("^","^{").replace("_","}_{").replace("\right)","}\right)"))
                                        
    
    del df['vminus'], df['vplus']
    for s in suff:
        del df[val+s]
    return df
                                
valout = [("t0", "$t_0$ (BJD)"),
          ("Eflare","$E_{f}$ (erg)"),
          ("ED_distr", "$ED$ (s)"),
          ("radius", "$\omega/2$ (deg)"),
          ("phi0","$\phi_0$ (deg)"),
          ("a","$a$"),
          ("i","$i$ (deg)"),
          ("fwhm", "FWHM (d)"),
          ("theta_a", "$\theta_f$ (deg)")]
cp = df.copy(deep=True)
for val, out in valout:
    print(val)
    cp = add_val_with_percentiles(df, val, out)
cp                               

t0
-5 POWER
3 POWERABS
0    29.152067
2     2.081736
4    28.260159
Name: vplus, dtype: float64
Eflare
32 POWER
34 POWERABS
ED_distr
2 POWER
4 POWERABS
0    16.877576
2    15.935516
4     2.633984
Name: vplus, dtype: float64
radius
0 POWER
1 POWERABS
0    0.472638
2    8.330115
4    0.537362
Name: vplus, dtype: float64
TRUE
phi0
0 POWER
2 POWERABS
0    1.219632
2    1.507483
4    1.507483
Name: vplus, dtype: float64
a
-1 POWER
0 POWERABS
0    7.413992
2    8.176406
4    0.377360
Name: vplus, dtype: float64
TRUE
i
0 POWER
1 POWERABS
0    0.480687
2    3.432163
4    3.432163
Name: vplus, dtype: float64
TRUE
fwhm
-4 POWER
-2 POWERABS
0    16.954826
2     1.320638
4    12.461185
Name: vplus, dtype: float64
theta_a
0 POWER
2 POWERABS
0    0.915509
2    9.933191
4    5.744893
Name: vplus, dtype: float64
TRUE


,date,ID,burnin,steps,walkers,nparam,$t_0$ (BJD),$E_{flare}$ (erg)$\cdot 10^{34}$,$ED$ (s),radius (deg),$\phi_0$ (deg),$a$,$i$ (deg),FWHM (d),$\theta_f$ (deg)
0,13_12_2019_10_18,100004076,3000,5000,32,6,$1358.51265\left(^{29}_{24}\right)$,$0.51\left(^{2}_{4}\right)$,$45333\left(^{17}_{33}\right)$,$19.4\left(^{5}_{7}\right)$,$-107\left(^{1}_{2}\right)$,$15.47\left(^{74}_{112}\right)$,$89.3\left(^{5}_{12}\right)$,$0.0372\left(^{17}_{12}\right)$,$73.1\left(^{9}_{15}\right)$
2,13_01_2020_11_28,237880881a,10000,11824,32,10,$1331.66407\left(^{2}_{2}\right)$,$6.89\left(^{141}_{146}\right)$,$7763\left(^{16}_{16}\right)$,$53.6\left(^{83}_{79}\right)$,$122\left(^{2}_{2}\right)$,$4.01\left(^{82}_{86}\right)$,$16.1\left(^{34}_{14}\right)$,$0.0244\left(^{1}_{1}\right)$,$31.8\left(^{99}_{64}\right)$
4,13_01_2020_11_28,237880881b,10000,11824,32,10,$1331.82458\left(^{28}_{18}\right)$,$4.83\left(^{23}_{31}\right)$,$5449\left(^{3}_{3}\right)$,$19.3\left(^{5}_{7}\right)$,$122\left(^{2}_{2}\right)$,$0.67\left(^{4}_{4}\right)$,$16.1\left(^{34}_{14}\right)$,$0.1027\left(^{12}_{13}\right)$,$46.8\left(^{57}_{31}\right)$


In [86]:
del cp['date']
del cp['nparam']
del cp['steps']
del cp['walkers']
del cp['burnin']

In [87]:
df3 = cp.copy()
nc = 'c' * (df3.shape[1]-2) #number of middle columns
footnote_pref= "\multicolumn{" + str(df3.shape[1]-2) + "}{l}" 
footnote_suf = "\n"
#fs = []
# for alpha, ref, repl in references:
#     f1 = footnote_pref + "{$^" + alpha + "$}{\citet\{" + ref + "}}" + footnote_suf
#     fs.append(f1)

stri = df3.to_latex(index=False,escape=False, column_format=f"l{nc}r")
stri = stri.replace("\\toprule","\hline")
stri = stri.replace("\\midrule","\hline")

# for alpha, ref, repl in references:
#     if len(repl) > 0:
#         for r in repl:
#             stri = stri.replace(r,r + r"$^" + alpha + "$")

#repl = " ".join(fs)
stri = stri.replace("\\bottomrule","\hline\n" )
print(stri)

\begin{tabular}{lccccccccr}
\hline
         ID &                          $t_0$ (BJD) & $E_{flare}$ (erg)$\cdot 10^{34}$ &                        $ED$ (s) &                   radius (deg) &               $\phi_0$ (deg) &                              $a$ &                      $i$ (deg) &                         FWHM (d) &               $\theta_f$ (deg) \\
\hline
  100004076 &  $1358.51265\left(^{29}_{24}\right)$ &      $0.51\left(^{2}_{4}\right)$ &  $45333\left(^{17}_{33}\right)$ &    $19.4\left(^{5}_{7}\right)$ &  $-107\left(^{1}_{2}\right)$ &  $15.47\left(^{74}_{112}\right)$ &   $89.3\left(^{5}_{12}\right)$ &  $0.0372\left(^{17}_{12}\right)$ &   $73.1\left(^{9}_{15}\right)$ \\
 237880881a &    $1331.66407\left(^{2}_{2}\right)$ &  $6.89\left(^{141}_{146}\right)$ &   $7763\left(^{16}_{16}\right)$ &  $53.6\left(^{83}_{79}\right)$ &   $122\left(^{2}_{2}\right)$ &    $4.01\left(^{82}_{86}\right)$ &  $16.1\left(^{34}_{14}\right)$ &    $0.0244\left(^{1}_{1}\right)$ &  $31.8\left(^{99}_{64}\

In [119]:
## Add references

In [120]:
with open(f"{CWD}/paper/draft/tables/lcs.tex", "w") as f:
    f.write(stri)